In [1]:
import pandas as pd

In [2]:
import os
os.chdir('/data/core-kind/fatemeh/codes/data_exploration')

In [24]:
opt = pd.read_csv('files/all_dicoms.csv', index_col=0)
len(opt)

43396

In [6]:
opt.head()

,Filename,PatientID,SOPClassUID,Modality,Laterality
1,1.2.276.0.75.2.2.42.50126425987.20231129154423...,KK-SCIVIAS-03474,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
16,1.2.276.0.75.2.2.42.50126425987.20220412143853...,KK-SCIVIAS-01172,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
21,1.2.276.0.75.2.2.42.50126425987.20220224114749...,KK-SCIVIAS-01048,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
29,1.2.276.0.75.2.2.42.50126425987.20220517150138...,KK-SCIVIAS-01354,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,R
42,1.2.276.0.75.2.2.42.50126425987.20240730100425...,KK-SCIVIAS-08103,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L


In [7]:
len(opt)

4576

In [9]:
opt.Filename.nunique()

4576

In [13]:
len(set(opt.index))

43396

In [5]:
opt.groupby(['SOPClassUID', 'Modality']).size()

SOPClassUID                       Modality
1.2.840.10008.5.1.4.1.1.66        OPT         34244
1.2.840.10008.5.1.4.1.1.77.1.5.1  OP           4576
1.2.840.10008.5.1.4.1.1.77.1.5.4  OPT          4576
dtype: int64

In [58]:
'1.2.276.0.75.2.2.42.50126425987.20231129154423733.6104471380.2.dcm' in opt.Filename.values

True

# debug process dicoms

In [26]:
from process_dicoms import *

In [45]:
processor = DicomProcessor('./new')

In [36]:
valid = '1.2.276.0.75.2.2.42.50126425987.20210203130203608.5664904690.2.dcm'

In [31]:
page = next(iter(processor.page_iterator))

In [40]:
if "Contents" in page:
    for obj in tqdm(page["Contents"], desc="Processing DICOM files"):
        key = obj["Key"]
        if key.lower().endswith(".dcm"):
            
            if os.path.basename(key) == valid:
                file_path = processor.download_file(key)
                break

Processing DICOM files:  99%|█████████▉| 991/1000 [00:01<00:00, 800.86it/s]


In [48]:
dicom_path = file_path
dicom = pydicom.dcmread(dicom_path)
meta = processor.extract_meta(dicom, dicom_path)

In [49]:
meta

{'Filename': '1.2.276.0.75.2.2.42.50126425987.20210203130203608.5664904690.2.dcm',
 'Status': 'Saved',
 'Modality': 'OPT',
 'SOPClassUID': '1.2.840.10008.5.1.4.1.1.77.1.5.4',
 'Laterality': 'R',
 'PatientName': 'KK-SCIVIAS-00031^GM'}

# check label file

In [15]:
ls ~/data

IDRiD_data/  OCTID/  SCIAVIAS/  oct_weights.pth  sct_labels.csv


In [11]:
df = pd.read_csv('~/data/sct_labels.csv', index_col=0)
df.head()

,studyid,disease_en,subdiscipline
0,KK-SCIVIAS-00003,Chronic ulcerative colitis,gastroenterology
1,KK-SCIVIAS-00004,Crohn's disease,gastroenterology
2,KK-SCIVIAS-00005,Crohn's disease,gastroenterology
3,KK-SCIVIAS-00006,Conjunctivitis,immunology/rheumatology
4,KK-SCIVIAS-00006,Periodic fever and aphthous stomatitis with ph...,immunology/rheumatology


In [34]:
len(df), df.studyid.nunique()

(2797, 2458)

In [35]:
df.disease_en.value_counts()

Control                                              387
Celiac disease                                       119
Cystic fibrosis                                      113
Crohn's disease                                       50
Diabetes mellitus type 1                              45
                                                    ... 
Arthritis of finger of bilateral hand                  1
Pyruvate dehydrogenase complex deficiency              1
Pain of knee joint                                     1
Joubert syndrome with congenital hepatic fibrosis      1
Chronic type B viral hepatitis                         1
Name: disease_en, Length: 691, dtype: int64

In [40]:
all_df = pd.read_csv('/data/core-kind/fatemeh/data/SCIAVIAS/all.csv')


In [41]:
len(all_df)

14996

In [42]:
all_df.groupby(['Modality', 'SOPClassUID', 'has_pixel_array']).size()

Modality  SOPClassUID                       has_pixel_array
OP        1.2.840.10008.5.1.4.1.1.77.1.5.1  True                1469
OPT       1.2.840.10008.5.1.4.1.1.66        False              12058
          1.2.840.10008.5.1.4.1.1.77.1.5.4  True                1469
dtype: int64

In [62]:
opt_df = pd.read_csv('/data/core-kind/fatemeh/data/SCIAVIAS/meta.csv')
len(opt_df), opt_df.Filename.nunique()

(1569, 1162)

In [48]:
opt_df.head()

,Unnamed: 0,Filename,Status,Modality,SOPClassUID,Laterality,PatientName,has_pixel_array,ndim,page
0,0,1.2.276.0.75.2.2.42.50126425987.20210118121227...,Saved,OPT,1.2.840.10008.5.1.4.1.1.77.1.5.4,R,KK-SCIVIAS-00004^JJ,True,3,0
1,1,1.2.276.0.75.2.2.42.50126425987.20210118121511...,Saved,OPT,1.2.840.10008.5.1.4.1.1.77.1.5.4,L,KK-SCIVIAS-00004^JJ,True,3,0
2,2,1.2.276.0.75.2.2.42.50126425987.20210120100105...,Saved,OPT,1.2.840.10008.5.1.4.1.1.77.1.5.4,R,KK-SCIVIAS-00011^GM,True,3,0
3,3,1.2.276.0.75.2.2.42.50126425987.20210120100628...,Saved,OPT,1.2.840.10008.5.1.4.1.1.77.1.5.4,L,KK-SCIVIAS-00011^GM,True,3,0
4,4,1.2.276.0.75.2.2.42.50126425987.20210120131545...,Saved,OPT,1.2.840.10008.5.1.4.1.1.77.1.5.4,R,KK-SCIVIAS-00010^GM,True,3,0


In [56]:
grouped = opt_df.groupby('Filename')
filtered = grouped.size()>1


array([ True,  True, False, ..., False, False, False])

In [58]:
df = opt_df
filtered_df = df[df['Filename'].isin(df['Filename'].value_counts()[df['Filename'].value_counts() > 1].index)]


In [61]:
df[df.Filename=='1.2.276.0.75.2.2.42.50126425987.20210118121227438.5984172510.2.dcm']

,Unnamed: 0,Filename,Status,Modality,SOPClassUID,Laterality,PatientName,has_pixel_array,ndim,page
0,0,1.2.276.0.75.2.2.42.50126425987.20210118121227...,Saved,OPT,1.2.840.10008.5.1.4.1.1.77.1.5.4,R,KK-SCIVIAS-00004^JJ,True,3,0
865,865,1.2.276.0.75.2.2.42.50126425987.20210118121227...,Saved,OPT,1.2.840.10008.5.1.4.1.1.77.1.5.4,R,KK-SCIVIAS-00004^JJ,True,3,9


# how many file we have labels?

In [14]:
labels = pd.read_csv('~/data/sct_labels.csv', index_col=0)
labels.head()

,studyid,disease_en,subdiscipline
0,KK-SCIVIAS-00003,Chronic ulcerative colitis,gastroenterology
1,KK-SCIVIAS-00004,Crohn's disease,gastroenterology
2,KK-SCIVIAS-00005,Crohn's disease,gastroenterology
3,KK-SCIVIAS-00006,Conjunctivitis,immunology/rheumatology
4,KK-SCIVIAS-00006,Periodic fever and aphthous stomatitis with ph...,immunology/rheumatology


In [15]:
len(opt), len(labels)

(4576, 2797)

In [17]:
opt.head()

,Filename,PatientID,SOPClassUID,Modality,Laterality
1,1.2.276.0.75.2.2.42.50126425987.20231129154423...,KK-SCIVIAS-03474,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
16,1.2.276.0.75.2.2.42.50126425987.20220412143853...,KK-SCIVIAS-01172,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
21,1.2.276.0.75.2.2.42.50126425987.20220224114749...,KK-SCIVIAS-01048,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
29,1.2.276.0.75.2.2.42.50126425987.20220517150138...,KK-SCIVIAS-01354,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,R
42,1.2.276.0.75.2.2.42.50126425987.20240730100425...,KK-SCIVIAS-08103,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L


In [20]:
meta = opt
meta = meta.rename(columns={'PatientID':'studyid'})
meta.head()

,Filename,studyid,SOPClassUID,Modality,Laterality
1,1.2.276.0.75.2.2.42.50126425987.20231129154423...,KK-SCIVIAS-03474,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
16,1.2.276.0.75.2.2.42.50126425987.20220412143853...,KK-SCIVIAS-01172,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
21,1.2.276.0.75.2.2.42.50126425987.20220224114749...,KK-SCIVIAS-01048,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
29,1.2.276.0.75.2.2.42.50126425987.20220517150138...,KK-SCIVIAS-01354,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,R
42,1.2.276.0.75.2.2.42.50126425987.20240730100425...,KK-SCIVIAS-08103,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L


In [21]:
joined = pd.merge(labels, meta, on='studyid', how='inner')
joined.head()

,studyid,disease_en,subdiscipline,Filename,SOPClassUID,Modality,Laterality
0,KK-SCIVIAS-00004,Crohn's disease,gastroenterology,1.2.276.0.75.2.2.42.50126425987.20210118121227...,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,R
1,KK-SCIVIAS-00004,Crohn's disease,gastroenterology,1.2.276.0.75.2.2.42.50126425987.20210118121511...,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
2,KK-SCIVIAS-00005,Crohn's disease,gastroenterology,1.2.276.0.75.2.2.42.50126425987.20210122124555...,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,R
3,KK-SCIVIAS-00005,Crohn's disease,gastroenterology,1.2.276.0.75.2.2.42.50126425987.20210122124939...,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
4,KK-SCIVIAS-00006,Conjunctivitis,immunology/rheumatology,1.2.276.0.75.2.2.42.50126425987.20210128123911...,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,R


In [22]:
len(joined)

4527

In [27]:
opt.head()

,PatientID,SOPClassUID,Modality,Laterality
Filename,,,,
1.2.276.0.75.2.2.42.50126425987.20220428110722762.2520207380.dcm,KK-SCIVIAS-01282,1.2.840.10008.5.1.4.1.1.66,OPT,R
1.2.276.0.75.2.2.42.50126425987.20231129154423733.6104471380.2.dcm,KK-SCIVIAS-03474,1.2.840.10008.5.1.4.1.1.77.1.5.4,OPT,L
1.2.276.0.75.2.2.42.50126425987.20220302120142421.3104907140.dcm,KK-SCIVIAS-01078,1.2.840.10008.5.1.4.1.1.66,OPT,L
1.2.276.0.75.2.2.42.50126425987.20210311093602786.5768164460.dcm,KK-SCIVIAS-00097,1.2.840.10008.5.1.4.1.1.66,OPT,L
1.2.276.0.75.2.2.42.50126425987.20240516151053067.3132111250.dcm,KK-SCIVIAS-01559,1.2.840.10008.5.1.4.1.1.66,OPT,L


In [29]:
print(1)

1


In [31]:
for f in opt.index:
    print(f)

1.2.276.0.75.2.2.42.50126425987.20220428110722762.2520207380.dcm
1.2.276.0.75.2.2.42.50126425987.20231129154423733.6104471380.2.dcm
1.2.276.0.75.2.2.42.50126425987.20220302120142421.3104907140.dcm
1.2.276.0.75.2.2.42.50126425987.20210311093602786.5768164460.dcm
1.2.276.0.75.2.2.42.50126425987.20240516151053067.3132111250.dcm
1.2.276.0.75.2.2.42.50126425987.20230725154600726.6068438630.dcm
1.2.276.0.75.2.2.42.50126425987.20210715114550504.3068699980.dcm
1.2.276.0.75.2.2.42.50126425987.20220912125542966.4616261290.dcm
1.2.276.0.75.2.2.42.50126425987.20240214161003874.3172825840.dcm
1.2.276.0.75.2.2.42.50126425987.20220420122928314.4808161180.dcm
1.2.276.0.75.2.2.42.50126425987.20230623115219081.6088785250.dcm
1.2.276.0.75.2.2.42.50126425987.20210907100119750.3904841130.1.dcm
1.2.276.0.75.2.2.42.50126425987.20220404110521505.6112392330.1.dcm
1.2.276.0.75.2.2.42.50126425987.20211220093954604.5812747750.dcm
1.2.276.0.75.2.2.42.50126425987.20220516114200175.70856631.1.dcm
1.2.276.0.75.2.2.42